Importando libraries

In [1]:
import pandas as pd
import numpy as np
import talib
from funcoes_preco import organize_data, get_efficiency_ratio, get_stats
import dwx_query
from datetime import datetime, timedelta

Dados de entrada

In [2]:
files_dir = 'C:/Users/guicr/AppData/Roaming/MetaQuotes/Terminal/'\
            'D0E8209F77C8CF37AD8BF550E51FF075/MQL5/Files/'
symbol = 'Bra50Feb22'
timeframes = ['M1', 'M5', 'M15']
start = datetime(2022, 1, 1).timestamp()
end = datetime(2022, 1, 29).timestamp()

Gravando dados recebidos em um dicionário

In [3]:
data = {timeframe: dwx_query.inform_historic_data(files_dir, symbol, timeframe, 
                                                  start, end) 
        for timeframe in timeframes}

Bra50Feb22_M1
historic_data: Bra50Feb22 M1 11280 bars
Bra50Feb22_M5
historic_data: Bra50Feb22 M5 2260 bars
Bra50Feb22_M15
historic_data: Bra50Feb22 M15 760 bars


Enviando os dados para dataframes

In [4]:
symbol_data = {timeframe: pd.DataFrame(data[timeframe], 
                                       columns=['Date', 'Open', 'High', 'Low', 
                                                'Close', 'Tick Volume', 
                                                'Real Volume'])
               for timeframe in timeframes}

Copiando os dataframes para uma variável de manipulação

In [5]:
eqty_table = symbol_data.copy()

Calculando o retorno barra a barra

In [6]:
pct_change = {timeframe: eqty_table[timeframe]['Close'].pct_change() 
              for timeframe in timeframes}
for timeframe in timeframes:
    eqty_table[timeframe]['Percent Change'] = pct_change[timeframe]

Triggers

In [7]:
cci_period = 20
cci = talib.CCI(eqty_table['M5']['High'], eqty_table['M5']['Low'], 
                eqty_table['M5']['Close'], timeperiod=cci_period)
eqty_table['M5']['CCI'] = cci

In [8]:
std_dev_period = 15
std_dev = get_stats(eqty_table['M5']['Close'], std_dev_period)['std']
eqty_table['M5']['Standard Deviation'] = std_dev

Filters

In [9]:
er_period = 8
efficiency_ratio = get_efficiency_ratio(eqty_table['M15']['Close'], 
                                        timeperiod=er_period)
eqty_table['M15']['Efficiency Ratio'] = efficiency_ratio['efficiency_ratio']


In [10]:
skew_period = 15
skewness = get_stats(eqty_table['M15']['Close'], skew_period)['skewness']
eqty_table['M15']['Skewness'] = skewness